## 1. Importing Libraries

In [ ]:
# Import libraries
import pandas as pd
import IPython.display as ipd
from matplotlib import pyplot as plt
import librosa
import librosa.display
import os
import shutil
import random
import numpy as np
from tqdm.notebook import tqdm
import noisereduce as nr
import sklearn.preprocessing

In [ ]:
# Set seed
np.random.seed(2022)

## 2. Loading Data

In [ ]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# Unzip audio zip file
# shutil.unpack_archive('Data/Swahili_words.zip', 'Swahili_words')

## 3. Data Exploration

In [ ]:
# Preview train set
train.head()

In [ ]:
# Preview test set
display(test.head(), train.shape, test.shape)

In [ ]:
# Target distribution
train.Swahili_word.value_counts()

In [ ]:
# Sample some words
for word in random.sample(train.Swahili_word.unique().tolist(), 6):
  sample = train[train.Swahili_word == word].Word_id.sample(1).values[0]
  display(word, sample, ipd.Audio('Swahili_words/'+ sample ))

## 4. Data Cleaning

1. **Load Audio Files:** It can load multiple audio files from a specified folder, allowing you to work with a collection of audio recordings.

2. **Visualize Audio Waveforms:** It visualizes the waveform of each loaded audio file using Matplotlib, which helps you inspect the raw audio data.

3. **Resample Audio:** It resamples the audio to a specified target sampling rate if the native sampling rate of the audio files differs from the target rate. This can be important when you need audio data to be at a consistent sampling rate.

4. **Optional Noise Reduction:** Although this code includes a placeholder for noise reduction using the `noisereduce` library, you can replace it with your own noise reduction techniques if needed. Noise reduction is particularly useful when dealing with audio recordings that contain background noise.

5. **Normalization:** It normalizes the audio data to ensure that it falls within a common range, typically between -1 and 1. Normalization can help make the audio data more suitable for analysis or modeling.

6. **Error Handling:** The code includes error handling to handle exceptions that may occur during audio processing, ensuring that the script doesn't crash if there are issues with specific audio files.

In [ ]:
# Load the dummy audio files
data_folder = r'Swahili_words'
audio_files = [os.path.join(data_folder, filename) for filename in os.listdir(data_folder)]  # get list of audio files
audio_files = [audio_file for audio_file in audio_files if audio_file.endswith('.wav')]  # filter out non-audio files
print(f'Found {len(audio_files)} files')  # print the number of files

# Limit the number of visualizations to 5
num_visualizations = 5
for i, audio_file in enumerate(audio_files[:num_visualizations]):
    try:
        y, sr = librosa.load(audio_file, sr=None)  # Load the audio file with its native sampling rate

        # Visualize the audio waveform
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f"Audio Waveform - File {i + 1}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.show()

        # Preprocessing steps
        target_sr = 16000  # Target sampling rate (e.g., 16 kHz)
        if sr != target_sr:
            y = librosa.resample(y, sr, target_sr)
            sr = target_sr

        # 2. Noise reduction (optional):
        # You can apply noise reduction techniques if the audio has background noise.
        y= nr.reduce_noise(y, sr=sr)
        
    
        # Example: y = some_noise_reduction_function(y)

        # 3. Normalization:
        # Normalize the audio to a common range, usually between -1 and 1.
        y = librosa.util.normalize(y)

        # 4. Plot the preprocessed audio waveform
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f"Preprocessed Audio Waveform - File {i + 1}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.show()

    except Exception as e:
        print(f"Error processing file: {audio_file}")
        print(e)

## Exploratory Data Analysis

In [ ]:
# Create a dictionary to check the audio files with the same transcription 
# Get three sample
dict_samples=dict()
for word in train['Swahili_word'].unique().tolist():
    sample = train[train['Swahili_word'] == word]['Word_id'].sample(3).values[:]
    dict_samples[word] = sample

In [ ]:
dict_samples

In [ ]:
# show three wavefiles for all words in time domain, for easy comparison between the words
for word in dict_samples:
    i=0
    fig, ax = plt.subplots(nrows=1, ncols=3, sharey=True)
    fig.set_size_inches(10, 5)
    fig.suptitle(word)
    for audiofile in dict_samples[word]:
        x, sr = librosa.load('Swahili_words/'+audiofile)
        img = librosa.display.waveshow(x, sr=sr, ax=ax[i])
        i+=1

In [ ]:
# show the three files of each word in frequency domain
for word in dict_samples:
    i=0
    fig, ax = plt.subplots(nrows=1, ncols=3, sharey=True)
    fig.set_size_inches(10, 5)
    fig.suptitle(word)
    for audiofile in dict_samples[word]:
        x, sr = librosa.load('Swahili_words/'+audiofile)
        X = librosa.amplitude_to_db(np.abs(librosa.stft(x)), ref=np.max)
        img = librosa.display.specshow(X, x_axis='time', sr=sr, ax=ax[i])
        i+=1